In [6]:
input_file = "data/20240518_1_HRM_ADC_FromMAX30101.csv"


In [7]:
# Run analysis CLI program in HRMAnalysisCPPCLI folder and called HRMAnalysisCPPCLI
# The program takes two params: the input file and the output file
import subprocess
subprocess.run(["./HRMAnalysisCPPCLI/HRMAnalysisCPPCLI", input_file, input_file + "_PROCESSED.csv"])


Output written to data/20240518_1_HRM_ADC_FromMAX30101.csv_PROCESSED.csv


CompletedProcess(args=['./HRMAnalysisCPPCLI/HRMAnalysisCPPCLI', 'data/20240518_1_HRM_ADC_FromMAX30101.csv', 'data/20240518_1_HRM_ADC_FromMAX30101.csv_PROCESSED.csv'], returncode=0)